<a href="https://colab.research.google.com/github/RyanTahnikoyev/windows-97-robidatathon/blob/main/robi_data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np
import glob
import math
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras import layers
import pandas as pd 

In [5]:
train_df=pd.read_csv("/content/drive/MyDrive/robi-datathon-2-pre-assessment/train.csv")
test_df=pd.read_csv("/content/drive/MyDrive/robi-datathon-2-pre-assessment/test.csv")

print(train_df["s52"].value_counts())
print(test_df["s52"].value_counts())
train_df["s52"][train_df["s52"]=="o"]="0"
train_df["s52"][train_df["s52"]=="l"]="1"
test_df["s52"][test_df["s52"]=="o"]="0"
test_df["s52"][test_df["s52"]=="l"]="1"

print(train_df["s52"].value_counts())

1    20524
l     6508
0     1094
o      196
Name: s52, dtype: int64
1    45569
l    36056
0     2811
o      629
Name: s52, dtype: int64
1    27032
0     1290
Name: s52, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value

In [6]:
for i in train_df.columns:
  print(train_df[i].value_counts())

b'gAAAAABinOicS09vrmgh0_JyEHihI13ptO0rCyHP7l76be71PWA2ReUc4HUQn16Fya1z8_VStNnFGaXJF262CgsuMPzOaknSeg=='    1
b'gAAAAABinOiekMHaC6-03yzvmLFVBfqljUIev5XFrbbJarEbo-mshNj5pWAXfTdmvEMQtb0WjtlboHt2rFBrCoipjAn9sOOZEg=='    1
b'gAAAAABinOiYtx-HpOZBYTtT-FndzIj6Nt2HTKu7UKAs-Dfxd8mxrvwxRNGE47Si_-kc5TiwVkTf59u94658aAU7gHD9-TGBMQ=='    1
b'gAAAAABinOia961WRLsDIRdEcGr5_RSKfmTjQ2ME5HBpIUtBdjJUeCTgVI6uzDIdnDRB58VBUvirHgdTdjgECltfpZ2XequVmA=='    1
b'gAAAAABinOibb7LcG8T4rQSnzf-b2GjK5D0F1ZLe6VMES-x90Pi5-Emk_dbp7xYXuepQpmVa_rhfEsemOUZUGn9_30mt-Vr8iQ=='    1
                                                                                                          ..
b'gAAAAABinOiawRN0qVxqLZn-_t1Qb87hktzzZnsBU00aUINCq1S48jnwhwxdn5XNd6mR189SFIILrWFbz0_LyBAvIlgUCImvqw=='    1
b'gAAAAABinOiZPy87pVtsRScbROxYGDhSan3Pp9q53S8zvzjZWJIhBd_PGqGRWUhxCwbTpmkda93liZMesGSchyhojqcCkJUD2w=='    1
b'gAAAAABinOib5XtfV1CgG7sJ3lqWVyVEX63543B09-SgMzYFlBkQuLmoEYkpfdNT1_v2JR0feyWOaSlNB_HLd2WSNncMF_Pu8Q=='    1
b'gAAAAABinOiZOg76i

#Preprocessing and Encoding
##Mispelled Columns:
##s52, s55
##Empty Columns:
##s56,s57,s59
##Oridinal Data Columns:
##n3,n15

In [7]:
missing_sum_row=np.sum(pd.isna(train_df),axis=1)

missing_sum_col=np.sum(pd.isna(train_df),axis=0)

In [8]:
missing_sum_row.value_counts()

5    22770
4     5270
3      282
dtype: int64

In [9]:
missing_col_map={i:j for i,j in zip(list(train_df.columns),missing_sum_col)}

In [10]:
len(missing_col_map)

36

In [11]:
for i,j in missing_col_map.items():
  if j>19000:
    train_df.drop(i,inplace=True,axis=1)

In [12]:
train_df.drop("id",inplace=True,axis=1)

#Ultimate labels for dataframe:
#['gender', 's11', 's12', 's13', 's16', 's17', 's18', 's48', 's52', 's53','s58', 's69', 's70', 's71', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7','n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15', 'label']

In [13]:
X_train_full=train_df.copy()

In [14]:
binary_cols= [cname for cname in X_train_full.columns if
              X_train_full[cname].nunique()==2 and cname!=X_train_full.columns[-1]]

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object" and cname 
                    not in binary_cols]
categorical_cols.extend(["n3","n15"])
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64'] and cname not in ["label","n3","n15"]]

In [15]:
def prepare_dataset(df,binary_cols,categorical_cols,numerical_cols):
  df=df.sample(frac=1)
  binary_df=df[binary_cols].copy()
  categorical_df=df[categorical_cols].copy()
  from sklearn.preprocessing import LabelEncoder
  for i in binary_df:
      le=LabelEncoder()
      binary_df[i]=le.fit_transform(binary_df[i])
  numerical_df=df[numerical_cols].copy()
  categorical_df=categorical_df.astype("O")
  categorical_df=pd.get_dummies(categorical_df)
  siamese_input_x=pd.concat([binary_df,categorical_df,numerical_df],axis=1)
  siamese_input_y=df["label"]
  return siamese_input_x,siamese_input_y


##Enconded Dataset

In [16]:
x,y=prepare_dataset(train_df,binary_cols,categorical_cols,numerical_cols)

In [17]:
import tensorflow as tf
import numpy as np
import cv2 
import os
import glob
import math
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras import layers
import pandas as pd 

#Basic Dense Model

In [18]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(x)

In [19]:
dense1= Dense(128,activation='relu')
dense2=Dense(256,activation="relu")
dense3=Dense(512,activation="relu")
dr2=layers.Dropout(0.5)
dense3_1=Dense(512,activation="relu")
dense4=Dense(256,activation="relu")
dense5=Dense(128,activation="relu")
dr3=layers.Dropout(0.5)
dense6=Dense(1024,activation="relu")
dense7=Dense(24,activation="relu")
dense8=Dense(1,activation="sigmoid")
dist=tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
model=tf.keras.Sequential([normalizer,dense1,dense2,dense3,dr2,dense3_1,dense4,dense5,dr3,dense6,dense7,dense8])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 66)               133       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               8576      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                        

In [ ]:
from sklearn.model_selection import StratifiedKFold


model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=1)
history_main=[]
kf=StratifiedKFold(n_splits=7,shuffle=True)
for i in range(1000):
  split=kf.split(x,y=y)
  print(i)
  for trainidx,testidx in split:
    print("train,test: ",str(len(trainidx)),str(len(testidx)))
    train_data=(x.iloc[trainidx],y[trainidx])
    test_data=(x.iloc[testidx],y[testidx])  
    history = model.fit(x=train_data[0],y=train_data[1],epochs=5,callbacks=[callback],validation_data=test_data)
  if history.history["val_accuracy"][0]>=0.98 and history.history["accuracy"][0]>=0.97:
    break 

0
train,test:  24276 4046
Epoch 1/5
759/759 [==============================] - 4s 4ms/step - loss: 0.4530 - accuracy: 0.8319 - val_loss: 0.4520 - val_accuracy: 0.8324
Epoch 2/5
759/759 [==============================] - 3s 4ms/step - loss: 0.4530 - accuracy: 0.8319 - val_loss: 0.4520 - val_accuracy: 0.8324
train,test:  24276 4046
Epoch 1/5
759/759 [==============================] - 3s 4ms/step - loss: 0.4504 - accuracy: 0.8335 - val_loss: 0.4681 - val_accuracy: 0.8223
Epoch 2/5
759/759 [==============================] - 3s 4ms/step - loss: 0.4503 - accuracy: 0.8335 - val_loss: 0.4684 - val_accuracy: 0.8223
train,test:  24276 4046
Epoch 1/5
759/759 [==============================] - 3s 4ms/step - loss: 0.4528 - accuracy: 0.8319 - val_loss: 0.4529 - val_accuracy: 0.8319
Epoch 2/5
759/759 [==============================] - 3s 4ms/step - loss: 0.4529 - accuracy: 0.8319 - val_loss: 0.4528 - val_accuracy: 0.8319
train,test:  24276 4046
Epoch 1/5
759/759 [==============================] - 3s 

In [ ]:
history.history

In [ ]:
embedding_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/robi-datathon-2-pre-assessment/model_weights/emb_weight_{epoch:04d}_{loss:.5f}.hdf5',
    save_weights_only=True,
    monitor='loss',
    mode='auto',
    save_best_only=True)

history = embedding_model.fit(siamese_input_x,siamese_input_y,batch_size=700,epochs=1000,callbacks=[model_checkpoint])

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')